## Using Komo for IK

In [19]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time

In [20]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

q_0 = C.getJointState()

In [21]:
obj = C.addFrame("object")
obj.setPosition([.3,0,.73])
#obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.ssBox, [.2,.05,0.04,0.01])
obj.setColor([1,1,0])
obj.setContact(1)


In [22]:
mk_obj = C.addFrame("mk_obj","object")
mk_obj.setShape(ry.ST.marker,[0.2])


In [23]:
mk_gripper = C.addFrame("mk_gripper","R_gripper")
mk_gripper.setShape(ry.ST.marker,[0.1])

mk_real = C.addFrame("mk_real_gripper")
mk_real.setShape(ry.ST.marker,[.1])


In [24]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex02_4.g")
S = RealWorld.simulation(ry.SimulatorEngine.bullet, True)
S.addSensor("camera")

In [25]:
#-- the following is the simulation loop
tau = .01

for t in range(200):
    time.sleep(0.01)
    q = S.get_q()
    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    C.setFrameState(RealWorld.getFrameState("obj1"),["object"])
    #send velocity controls to the simulation
    S.step(np.zeros_like(q), tau, ry.ControlMode.velocity)

In [26]:


# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,30, 5., True)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripper", "object"], ry.OT.sos, [1e2])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([.7,1.], ry.FS.position, ["R_gripper"], ry.OT.eq, [1e2],[-.0,-.1,-.0], order=2)
komo.addObjective([], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2],[-.0])
komo.addObjective([1.], 
                ry.FS.scalarProductXX, 
                ["object","R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [0]);
komo.addObjective([1.], 
                ry.FS.scalarProductZZ, 
                ["object","R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [1]);
print(komo.getT(),komo.getPathFrames().shape)

komo.optimize()
#komo.getReport()
#V = komo.view_play(False, 1)



30 (30, 101, 7)


In [27]:


for frame,tau in zip(komo.getPathFrames(),komo.getPathTau()):
    time.sleep(tau)
    
    C.setFrameState(frame)

    q= C.getJointState()

    #send position to the simulation
    S.step(q, tau, ry.ControlMode.position)